In [1]:
# s3 특정 경로에 있는 파일 이름 리딩
# 정재 후 폴더 생성

In [4]:
import sys
print(sys.path)
print(sys.executable)

['c:\\ProgramData\\anaconda3\\python313.zip', 'c:\\ProgramData\\anaconda3\\DLLs', 'c:\\ProgramData\\anaconda3\\Lib', 'c:\\ProgramData\\anaconda3', '', 'C:\\Users\\USER\\AppData\\Roaming\\Python\\Python313\\site-packages', 'c:\\ProgramData\\anaconda3\\Lib\\site-packages', 'c:\\ProgramData\\anaconda3\\Lib\\site-packages\\win32', 'c:\\ProgramData\\anaconda3\\Lib\\site-packages\\win32\\lib', 'c:\\ProgramData\\anaconda3\\Lib\\site-packages\\Pythonwin']
c:\ProgramData\anaconda3\python.exe


In [4]:
import boto3
import re

# S3 클라이언트 생성
s3 = boto3.client("s3")

bucket_name = "schoolfriends-bym"

# 버킷 내 모든 객체 가져오기
response = s3.list_objects_v2(Bucket=bucket_name, Prefix="data/")

ModuleNotFoundError: No module named 'boto3'

In [ ]:
for obj in response.get("Contents", []):
    key = obj["Key"]
    
    # CSV 파일만 처리
    if key.lower().endswith(".csv"):
        match = re.search(r"_([A-Z]+)__", key)
        if match:
            folder = match.group(1).lower()  # 소문자로 변환해서 폴더 이름으로
            new_key = f"{folder}/{key.split('/')[-1]}"
            
            print(f"Moving {key} -> {new_key}")
            
            # S3 내에서 파일 복사
            s3.copy_object(
                Bucket=bucket_name,
                CopySource={"Bucket": bucket_name, "Key": key},
                Key=new_key
            )
            
            # 원본 삭제
            s3.delete_object(Bucket=bucket_name, Key=key)